In [ ]:
!pip install roboflow ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
rf = Roboflow(api_key="ZVgrAS59nFr8o1TrBNvR")
project = rf.workspace("sonu-frtzd").project("hand-detection-5gzvh-ntp0h")
dataset = project.version(1).download("yolov8")
dataset_version = 1
# -----------------

dataset = project.version(dataset_version).download("yolov8")

print(f"Dataset downloaded successfully to: {dataset.location}")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to hand-detection-1 in yolov8:: 100%|██████████| 16034/16034 [00:04<00:00, 3233.88it/s]


Dataset downloaded successfully to: /content/hand-detection-1


In [ ]:
import os
import glob

# Path to the dataset you just downloaded
dataset_root = '/content/hand-detection-1'
splits = ['train', 'test', 'valid']

print("Starting to merge all classes into '0'...")
fixed_file_count = 0

for split in splits:
    label_dir = os.path.join(dataset_root, split, 'labels')
    if not os.path.exists(label_dir):
        print(f"Skipping {split} (not found)")
        continue

    label_files = glob.glob(os.path.join(label_dir, '*.txt'))

    for file_path in label_files:
        new_lines = []
        try:
            with open(file_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if len(parts) > 0:
                    # Check if the class is 0, 1, or 2
                    if parts[0] in ['0', '1', '2']:
                        # Change it to '0'
                        parts[0] = '0'
                        new_lines.append(" ".join(parts) + "\n")
                    # If it's something else, we ignore it

            # Overwrite the file with the merged labels
            with open(file_path, 'w') as f:
                f.writelines(new_lines)

            fixed_file_count += 1
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

print(f"Done! Merged classes in {fixed_file_count} label files.")

Starting to merge all classes into '0'...
Done! Merged classes in 8011 label files.


In [ ]:
import os
import glob
from pathlib import Path

# Path to the dataset folder after labels have been merged
dataset_root = '/content/hand-detection-1'
splits_to_check = ['train', 'test', 'valid']

print(f"--- Running Sanity Check on {dataset_root} ---")

grand_total_images = 0
grand_total_hands_images = 0
grand_total_empty_images = 0
grand_total_hand_annotations = 0

for split in splits_to_check:
    label_dir = Path(dataset_root) / split / 'labels'
    if not label_dir.exists():
        print(f"\nSkipping {split} (folder not found)")
        continue

    # Get all label files
    label_files = glob.glob(str(label_dir / '*.txt'))

    total_images_in_split = len(label_files)
    images_with_hands = 0
    empty_images = 0
    total_hand_annotations = 0

    if total_images_in_split == 0:
        print(f"\nNo label files found in {split}")
        continue

    for label_path_str in label_files:
        # Check if file is empty
        if os.path.getsize(label_path_str) == 0:
            empty_images += 1
        else:
            images_with_hands += 1
            # Count the lines (each line is one hand)
            with open(label_path_str, 'r') as f:
                total_hand_annotations += len(f.readlines())

    # Print results for this split
    print(f"\nResults for '{split}' split:")
    print(f"  Total Images Scanned: {total_images_in_split}")
    print(f"  Images with Hands:    {images_with_hands}")
    print(f"  Empty/BG Images:      {empty_images}")
    print(f"  Total Hand Annotations: {total_hand_annotations}")

    # Add to grand totals
    grand_total_images += total_images_in_split
    grand_total_hands_images += images_with_hands
    grand_total_empty_images += empty_images
    grand_total_hand_annotations += total_hand_annotations

print("\n-------------------------------------")
print("--- Grand Totals (All Splits) ---")
print(f"  Total Images: {grand_total_images}")
print(f"  Images with Hands:    {grand_total_hands_images}")
print(f"  Empty/BG Images:      {grand_total_empty_images}")
print(f"  Total Hand Annotations: {grand_total_hand_annotations}")
print("-------------------------------------")

if grand_total_empty_images > 0:
    print(f"\nNOTE: You have {grand_total_empty_images} background images.")
    print("The re-splitting script (Cell 3) will correctly shuffle these too,")
    print("which is good for training.")

--- Running Sanity Check on /content/hand-detection-1 ---

Results for 'train' split:
  Total Images Scanned: 7805
  Images with Hands:    6604
  Empty/BG Images:      1201
  Total Hand Annotations: 6650

Results for 'test' split:
  Total Images Scanned: 70
  Images with Hands:    67
  Empty/BG Images:      3
  Total Hand Annotations: 68

Results for 'valid' split:
  Total Images Scanned: 136
  Images with Hands:    133
  Empty/BG Images:      3
  Total Hand Annotations: 136

-------------------------------------
--- Grand Totals (All Splits) ---
  Total Images: 8011
  Images with Hands:    6804
  Empty/BG Images:      1207
  Total Hand Annotations: 6854
-------------------------------------

NOTE: You have 1207 background images.
The re-splitting script (Cell 3) will correctly shuffle these too,
which is good for training.


In [ ]:
import os
import glob
import shutil
import random
from pathlib import Path

# --- Configuration ---
source_dir = Path('/content/hand-detection-1')
output_dir = Path('/content/hand-dataset-FINAL')
splits = {'train': 0.75, 'valid': 0.12, 'test': 0.13} # 75% + 12% + 13% = 100%
# ---------------------

all_image_paths = []
print("Collecting all images...")

# Find all images in the original train, test, and valid folders
for split in ['train', 'test', 'valid']:
    image_search_path = str(source_dir / split / 'images' / '*.*')
    all_image_paths.extend([Path(p) for p in glob.glob(image_search_path)])

# Shuffle them randomly
random.shuffle(all_image_paths)

print(f"Found {len(all_image_paths)} total images. Shuffling...")

# Create new directory structure
for split_name in splits:
    (output_dir / split_name / 'images').mkdir(parents=True, exist_ok=True)
    (output_dir / split_name / 'labels').mkdir(parents=True, exist_ok=True)

# Calculate split points
total_files = len(all_image_paths)
train_end = int(total_files * splits['train'])
valid_end = train_end + int(total_files * splits['valid'])

# Copy files to new destinations
print("Copying files to new balanced dataset...")
for i, img_path in enumerate(all_image_paths):

    # Find the matching label file
    label_path = source_dir / img_path.parent.parent.name / 'labels' / f"{img_path.stem}.txt"

    # Determine which new split this file belongs to
    if i < train_end:
        current_split = 'train'
    elif i < valid_end:
        current_split = 'valid'
    else:
        current_split = 'test'

    # Define new destination paths
    dest_img_path = output_dir / current_split / 'images' / img_path.name
    dest_label_path = output_dir / current_split / 'labels' / f"{img_path.stem}.txt"

    # Copy the files
    if label_path.exists():
        shutil.copy(img_path, dest_img_path)
        shutil.copy(label_path, dest_label_path)
    else:
        # This handles the background images (which have no label file)
        shutil.copy(img_path, dest_img_path)

print("\n--- Dataset Re-split Complete! ---")
print(f"New dataset is at: {output_dir}")
print(f"Train: {len(list((output_dir / 'train' / 'images').glob('*')))} files")
print(f"Valid: {len(list((output_dir / 'valid' / 'images').glob('*')))} files")
print(f"Test:  {len(list((output_dir / 'test' / 'images').glob('*')))} files")

Found 8011 total images. Shuffling...
Copying files to new balanced dataset...

--- Dataset Re-split Complete! ---
New dataset is at: /content/hand-dataset-FINAL
Train: 6008 files
Valid: 961 files
Test:  1042 files


In [ ]:
%%writefile /content/hand-dataset-FINAL/data.yaml
names:
  0: hand
nc: 1
train: /content/hand-dataset-FINAL/train/images
val: /content/hand-dataset-FINAL/valid/images
test: /content/hand-dataset-FINAL/test/images

Writing /content/hand-dataset-FINAL/data.yaml


dataset 2

In [ ]:
API_KEY = "ZVgrAS59nFr8o1TrBNvR"

# Your new project details
WORKSPACE_ID = "sonu-frtzd"
PROJECT_ID = "hands-lakl8-lyl74"
DATASET_VERSION = 1
# ---------------------

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE_ID).project(PROJECT_ID)
dataset = project.version(DATASET_VERSION).download("yolov8")

print(f"Dataset downloaded successfully to: {dataset.location}")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Hands-1 in yolov8:: 100%|██████████| 9612/9612 [00:01<00:00, 7029.82it/s]


Dataset downloaded successfully to: /content/Hands-1


In [ ]:
import glob
from pathlib import Path

# This path comes from Cell 1
dataset_root = dataset.location
splits = ['train', 'test', 'valid']

print(f"Starting to merge all classes in {dataset_root}...")
fixed_file_count = 0

for split in splits:
    label_dir = os.path.join(dataset_root, split, 'labels')
    if not os.path.exists(label_dir):
        print(f"Skipping {split} (not found)")
        continue

    label_files = glob.glob(os.path.join(label_dir, '*.txt'))

    for file_path in label_files:
        new_lines = []
        try:
            with open(file_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if len(parts) > 0:
                    # Check if the class is 0, 1, 2, or 3
                    if parts[0] in ['0', '1', '2', '3']:
                        # Change it to '0'
                        parts[0] = '0'
                        new_lines.append(" ".join(parts) + "\n")

            # Overwrite the file with the merged labels
            with open(file_path, 'w') as f:
                f.writelines(new_lines)

            fixed_file_count += 1
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

print(f"Done! Merged classes in {fixed_file_count} label files.")

Starting to merge all classes in /content/Hands-1...
Done! Merged classes in 4800 label files.


In [ ]:
# This path comes from Cell 1
dataset_root = dataset.location
splits_to_check = ['train', 'test', 'valid']

print(f"--- Running Sanity Check on {dataset_root} ---")

grand_total_images = 0
grand_total_hands_images = 0
grand_total_empty_images = 0
grand_total_hand_annotations = 0

for split in splits_to_check:
    label_dir = Path(dataset_root) / split / 'labels'
    if not label_dir.exists():
        print(f"\nSkipping {split} (folder not found)")
        continue

    label_files = glob.glob(str(label_dir / '*.txt'))
    total_images_in_split = len(label_files)
    images_with_hands = 0
    empty_images = 0
    total_hand_annotations = 0

    for label_path_str in label_files:
        if os.path.getsize(label_path_str) == 0:
            empty_images += 1
        else:
            images_with_hands += 1
            with open(label_path_str, 'r') as f:
                total_hand_annotations += len(f.readlines())

    print(f"\nResults for '{split}' split:")
    print(f"  Total Images Scanned: {total_images_in_split}")
    print(f"  Images with Hands:    {images_with_hands}")
    print(f"  Empty/BG Images:      {empty_images}")
    print(f"  Total Hand Annotations: {total_hand_annotations}")

    grand_total_images += total_images_in_split
    grand_total_hands_images += images_with_hands
    grand_total_empty_images += empty_images
    grand_total_hand_annotations += total_hand_annotations

print("\n-------------------------------------")
print("--- Grand Totals (All Splits) ---")
print(f"  Total Images: {grand_total_images}")
print(f"  Images with Hands:    {grand_total_hands_images}")
print(f"  Empty/BG Images:      {grand_total_empty_images}")
print(f"  Total Hand Annotations: {grand_total_hand_annotations}")
print("-------------------------------------")

# You should see ~15,044 hand annotations
# (2559 + 3418 + 4560 + 4507 = 15044)

--- Running Sanity Check on /content/Hands-1 ---

Results for 'train' split:
  Total Images Scanned: 3840
  Images with Hands:    3830
  Empty/BG Images:      10
  Total Hand Annotations: 12015

Results for 'test' split:
  Total Images Scanned: 480
  Images with Hands:    479
  Empty/BG Images:      1
  Total Hand Annotations: 1514

Results for 'valid' split:
  Total Images Scanned: 480
  Images with Hands:    479
  Empty/BG Images:      1
  Total Hand Annotations: 1515

-------------------------------------
--- Grand Totals (All Splits) ---
  Total Images: 4800
  Images with Hands:    4788
  Empty/BG Images:      12
  Total Hand Annotations: 15044
-------------------------------------


In [ ]:
%%writefile /content/Hands-1/data.yaml
names:
  0: hand
nc: 1
train: /content/Hands-1/train/images
val: /content/Hands-1/valid/images
test: /content/Hands-1/test/images

Overwriting /content/Hands-1/data.yaml


datset3

In [ ]:
WORKSPACE_ID = "sonu-frtzd"
PROJECT_ID = "hand-bo3ot-jhr1r"
DATASET_VERSION = 1 # Change this if your version is different
# ---------------------

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE_ID).project(PROJECT_ID)
dataset = project.version(DATASET_VERSION).download("yolov8")

# This is the path to the raw, downloaded dataset
print(f"Dataset downloaded successfully to: {dataset.location}")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to hand-1 in yolov8:: 100%|██████████| 10598/10598 [00:01<00:00, 7936.98it/s]


Dataset downloaded successfully to: /content/hand-1


In [ ]:
import glob
import os

# This path comes from Cell 1
dataset_root = dataset.location
splits = ['train', 'test', 'valid']

# Create a list of all possible class IDs (0...25)
all_classes = [str(i) for i in range(26)]

print(f"Starting to merge all 26 classes in {dataset_root}...")
fixed_file_count = 0

for split in splits:
    label_dir = os.path.join(dataset_root, split, 'labels')
    if not os.path.exists(label_dir):
        print(f"Skipping {split} (not found)")
        continue

    label_files = glob.glob(os.path.join(label_dir, '*.txt'))

    for file_path in label_files:
        new_lines = []
        try:
            with open(file_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if len(parts) > 0:
                    # Check if the class ID is one of the 26
                    if parts[0] in all_classes:
                        # Change it to '0'
                        parts[0] = '0'
                        new_lines.append(" ".join(parts) + "\n")

            # Overwrite the file with the merged labels
            with open(file_path, 'w') as f:
                f.writelines(new_lines)

            fixed_file_count += 1
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

print(f"Done! Merged classes in {fixed_file_count} label files.")

Starting to merge all 26 classes in /content/hand-1...
Done! Merged classes in 5293 label files.


In [ ]:
import shutil
import random
from pathlib import Path

# --- Configuration ---
# 'dataset.location' comes from Cell 1
source_dir = Path(dataset.location)
output_dir = Path('/content/hand-dataset-3') # Your new folder name
subsample_percentage = 0.50 # Keep 50%
splits_to_process = ['train', 'test', 'valid']
# ---------------------

print(f"Creating new 50% subsampled dataset at {output_dir}...")

for split in splits_to_process:
    # Define source and destination paths
    source_img_dir = source_dir / split / 'images'
    source_label_dir = source_dir / split / 'labels'

    output_img_dir = output_dir / split / 'images'
    output_label_dir = output_dir / split / 'labels'

    # Create new output directories
    output_img_dir.mkdir(parents=True, exist_ok=True)
    output_label_dir.mkdir(parents=True, exist_ok=True)

    # Get a list of all images in the source split
    all_images = list(source_img_dir.glob('*.*'))
    random.shuffle(all_images)

    # Calculate how many to keep
    num_to_keep = int(len(all_images) * subsample_percentage)
    files_to_copy = all_images[:num_to_keep]

    print(f"Processing '{split}': Keeping {num_to_keep} of {len(all_images)} images.")

    # Copy the selected files
    for img_path in files_to_copy:
        label_path = source_label_dir / f"{img_path.stem}.txt"

        # Copy image
        shutil.copy(img_path, output_img_dir / img_path.name)

        # Copy label file *if it exists*
        if label_path.exists():
            shutil.copy(label_path, output_label_dir / label_path.name)

print("\n--- Subsampling Complete! ---")
print(f"New dataset is at: {output_dir}")

Creating new 50% subsampled dataset at /content/hand-dataset-3...
Processing 'train': Keeping 1698 of 3396 images.
Processing 'test': Keeping 358 of 717 images.
Processing 'valid': Keeping 590 of 1180 images.

--- Subsampling Complete! ---
New dataset is at: /content/hand-dataset-3


In [ ]:
%%writefile /content/hand-dataset-3/data.yaml
names:
  0: hand
nc: 1
train: /content/hand-dataset-3/train/images
val: /content/hand-dataset-3/valid/images
test: /content/hand-dataset-3/test/images

Writing /content/hand-dataset-3/data.yaml


data 4

In [ ]:
WORKSPACE_ID = "sonu-frtzd"
PROJECT_ID = "hand-7hx79-rxqsz"
DATASET_VERSION = 1 # Change this if your version is different
# ---------------------

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE_ID).project(PROJECT_ID)
dataset = project.version(DATASET_VERSION).download("yolov8")

# This is the path to the raw, downloaded dataset
# We will use this 'dataset.location' variable in the next cells
print(f"Dataset 4 downloaded successfully to: {dataset.location}")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to Hand-1 in yolov8:: 100%|██████████| 8976/8976 [00:01<00:00, 4530.22it/s]


Dataset 4 downloaded successfully to: /content/Hand-1


In [ ]:
import glob
import os
from pathlib import Path

# This path comes from Cell 1
dataset_root = dataset.location
splits = ['train', 'test', 'valid']

# Create a list of all 19 class IDs (0...18)
all_classes = [str(i) for i in range(19)]

print(f"Starting to merge all 19 classes in {dataset_root}...")
fixed_file_count = 0

for split in splits:
    label_dir = os.path.join(dataset_root, split, 'labels')
    if not os.path.exists(label_dir):
        print(f"Skipping {split} (not found)")
        continue

    label_files = glob.glob(os.path.join(label_dir, '*.txt'))

    for file_path in label_files:
        new_lines = []
        try:
            with open(file_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if len(parts) > 0:
                    # Check if the class ID is one of the 19
                    if parts[0] in all_classes:
                        # Change it to '0'
                        parts[0] = '0'
                        new_lines.append(" ".join(parts) + "\n")

            # Overwrite the file with the merged labels
            with open(file_path, 'w') as f:
                f.writelines(new_lines)

            fixed_file_count += 1
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

print(f"Done! Merged classes in {fixed_file_count} label files.")

Starting to merge all 19 classes in /content/Hand-1...
Done! Merged classes in 4482 label files.


In [ ]:
# This path comes from Cell 1
dataset_root = dataset.location

yaml_content = f"""
names:
  0: hand
nc: 1
train: {dataset_root}/train/images
val: {dataset_root}/valid/images
test: {dataset_root}/test/images
"""

with open(os.path.join(dataset_root, 'data.yaml'), 'w') as f:
    f.write(yaml_content)

print(f"Created data.yaml for {dataset_root}")

Created data.yaml for /content/Hand-1


In [ ]:
import shutil
from pathlib import Path

print("--- Starting to merge ALL FOUR datasets ---")

# 1. Define all your source datasets
# (The 'dataset.location' variable is from Cell 1)
source_datasets = [
    '/content/hand-dataset-FINAL',
    '/content/Hands-1',
    '/content/hand-dataset-3',
    dataset.location  # This is your new (e.g., /content/hand-7hx79-rxqsz-1)
]

# 2. Define the new Super-Dataset directory
output_root = Path('/content/Super-Dataset')
# Clear it out if it already exists from a previous run
if output_root.exists():
    shutil.rmtree(output_root)

# 3. Loop through each split and merge
splits_to_merge = ['train', 'valid', 'test']

for split in splits_to_merge:
    output_img_dir = output_root / split / 'images'
    output_label_dir = output_root / split / 'labels'
    output_img_dir.mkdir(parents=True, exist_ok=True)
    output_label_dir.mkdir(parents=True, exist_ok=True)

    print(f"\nMerging '{split}' files...")

    for source_root in source_datasets:
        if source_root is None:
            continue

        print(f"  > Copying from {source_root}")
        source_img_dir = Path(source_root) / split / 'images'
        source_label_dir = Path(source_root) / split / 'labels'

        # Use shell `cp` for speed
        if source_img_dir.exists():
            os.system(f'cp {source_img_dir}/*.* {output_img_dir}/')

        if source_label_dir.exists():
            os.system(f'cp {source_label_dir}/*.txt {output_label_dir}/')

print("\n--- Merge Complete! ---")

--- Starting to merge ALL FOUR datasets ---

Merging 'train' files...
  > Copying from /content/hand-dataset-FINAL
  > Copying from /content/Hands-1
  > Copying from /content/hand-dataset-3
  > Copying from /content/Hand-1

Merging 'valid' files...
  > Copying from /content/hand-dataset-FINAL
  > Copying from /content/Hands-1
  > Copying from /content/hand-dataset-3
  > Copying from /content/Hand-1

Merging 'test' files...
  > Copying from /content/hand-dataset-FINAL
  > Copying from /content/Hands-1
  > Copying from /content/hand-dataset-3
  > Copying from /content/Hand-1

--- Merge Complete! ---


In [ ]:
from pathlib import Path
print("\n--- Super-Dataset Image Count (Sanity Check) ---")

output_root = Path('/content/Super-Dataset')
train_count = len(list((output_root / 'train' / 'images').glob('*.*')))
valid_count = len(list((output_root / 'valid' / 'images').glob('*.*')))
test_count = len(list((output_root / 'test' / 'images').glob('*.*')))
total_count = train_count + valid_count + test_count

print(f"  Total Training Images: {train_count}")
print(f"  Total Validation Images: {valid_count}")
print(f"  Total Test Images:     {test_count}")
print(f"  -----------------------------------")
print(f"  GRAND TOTAL IMAGES:    {total_count}")


--- Super-Dataset Image Count (Sanity Check) ---
  Total Training Images: 14862
  Total Validation Images: 2480
  Total Test Images:     2597
  -----------------------------------
  GRAND TOTAL IMAGES:    19939


In [ ]:
%%writefile /content/Super-Dataset/data.yaml
names:
  0: hand
nc: 1
train: /content/Super-Dataset/train/images
val: /content/Super-Dataset/valid/images
test: /content/Super-Dataset/test/images

Writing /content/Super-Dataset/data.yaml


In [ ]:
from ultralytics import YOLO

# 1. Load a base model
model = YOLO("yolov8n.pt")

# 2. Define the path to your FINAL data.yaml file
data_yaml_path = '/content/Super-Dataset/data.yaml'

print(f"\n--- Starting Final Training on Super-Dataset ---")
print(f"   Epochs: 15")
print(f"   Batch Size: 32")
print(f"   Dataset: {data_yaml_path}")

# 3. Start training!
results = model.train(data=data_yaml_path,
                      epochs=20,
                      batch=64,
                      imgsz=640)

print("\n--- FINAL TRAINING COMPLETE ---")
print("Your new, super-robust model is saved in '/content/runs/detect/train/weights/best.pt'")


--- Starting Final Training on Super-Dataset ---
   Epochs: 15
   Batch Size: 32
   Dataset: /content/Super-Dataset/data.yaml
Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Super-Dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, 

In [ ]:
!zip -r /content/runs.zip /content/runs
from google.colab import files
files.download('/content/runs.zip')


  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/train/ (stored 0%)
  adding: content/runs/detect/train/args.yaml (deflated 53%)
  adding: content/runs/detect/train/confusion_matrix_normalized.png (deflated 37%)
  adding: content/runs/detect/train/confusion_matrix.png (deflated 35%)
  adding: content/runs/detect/train/val_batch0_pred.jpg (deflated 15%)
  adding: content/runs/detect/train/BoxPR_curve.png (deflated 27%)
  adding: content/runs/detect/train/val_batch2_pred.jpg (deflated 8%)
  adding: content/runs/detect/train/train_batch2331.jpg (deflated 8%)
  adding: content/runs/detect/train/BoxR_curve.png (deflated 17%)
  adding: content/runs/detect/train/val_batch2_labels.jpg (deflated 8%)
  adding: content/runs/detect/train/train_batch2.jpg (deflated 3%)
  adding: content/runs/detect/train/val_batch0_labels.jpg (deflated 15%)
  adding: content/runs/detect/train/results.png (deflated 7%)
  adding: content/runs/detect/train/re

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>